### Avaliação de Sistemas de RI

### Importando-se os pacotes necessários para a atividade

In [1]:
import pandas as pd
import re
import operator
from math import log
from collections import Counter
from unicodedata import normalize

### Obtendo-se a coleção

In [2]:
data_frame = pd.read_csv("results.csv")
documentos = data_frame["text"].tolist()

### Gerando-se os índices invertidos

#### Algoritmo "parse" para obter os tokens de um determinado documento

In [4]:
def is_pontuacao(token):
    '''
        Retorna-se um booleano indicando se o token passado por parâmetro é uma pontuação.
    '''
    lista_pontuacoes = [",", ".", "!", "?", ":", "/", "#", "*", "(", ")", ";", ""]
    return token.strip() in lista_pontuacoes

def remove_acentuacoes(token):
    '''
        Remove as acentuações de um token.
    '''
    token_unicode = unicode(token, "utf-8")
    return normalize('NFKD', token_unicode).encode('ASCII','ignore')

def parse(texto):
    '''
        Transforma o texto em uma lista de tokens, eliminando espaços vazios, acentuações e pontuações, e
        tranforma todas as palavras em letras minúsculas.
    '''
    texto = re.sub(r'[,.!?:*();]', ' ', str(texto))
    texto = re.sub(r'\n', ' ', texto)
    lista_texto = texto.split(' ')

    tokens = []
    for palavra in lista_texto:
        token = palavra.lower()
        token = token.strip()
        token = remove_acentuacoes(token)

        if(not is_pontuacao(token)):
            tokens.append(token)
    
    return tokens

#### Algoritmo para obtenção dos índices invertidos com frequência (IDF)

In [5]:
def get_indices_invertidos_com_frequencia(documentos):
    '''
        Obtém uma lista de índices invertidos a partir de uma lista de documentos,
        com a frequência do índice em cada um desses documentos.
    '''
    indices_invertidos = {}
    for i in range(len(documentos)):
        doc = parse(documentos[i])
        documento_counter = Counter(doc)
        
        for indice in documento_counter.keys():
            if(len(indice) > 3):
                if(indice in indices_invertidos):
                    indices_invertidos[indice][i] = documento_counter[indice]
                else:
                    indices_invertidos[indice] = { i: documento_counter[indice] }
    
    return indices_invertidos

#### Obtendo-se os índices com IDF

In [46]:
indices_invertidos_com_frequencia = get_indices_invertidos_com_frequencia(documentos)

### Implementação das seguintes versões do modelo vetorial:

    1. Representação binária;
    2. TF (lembre-se que esse modelo já está implementado);
    3. TF-IDF;
    4. BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

#### Algoritmos úteis para as implementações

In [8]:
def get_tuplas_documentos_ordenados_peso(dic_documentos):
    '''
        Obtém-se tuplas de documentos ordenados decrescentemente pelo peso, a partir de um dicionário
        que tem como chave o documento, e valor o peso.
    '''
    documentos_tuplas = dic_documentos.items()
    indice_peso = 1
    documentos_tuplas.sort(key = operator.itemgetter(indice_peso), reverse=True)
    
    return documentos_tuplas

def get_K_primeiros_ids_documentos(documentos_tuplas, k):
    '''
        Obtendo-se a lista de id dos k primeiros documentos, a partir de uma lista de tuplas, as quais
        o primeiro índice é o id do documento e o segundo é o seu peso.
    '''
    id_documentos_ordenados = [id_documento for id_documento, peso in documentos_tuplas]
    k_primeiros_id_documentos = id_documentos_ordenados[:k]
    
    return k_primeiros_id_documentos

def idf(quantidade_documentos_colecao, quantidade_documentos_com_palavra):
    '''
        Calcula o Inverse Document Frequency (IDF).
    '''
    return log((quantidade_documentos_colecao + 1) / quantidade_documentos_com_palavra)

#### Implementação do algoritmo para a *Representação Binária*

In [9]:
def get_documentos_ordem_representacao_binaria(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Representação Binária",
        na qual, os documentos que possuem a maior quantidade de termos da consultas
        são os mais representativos.
    '''
    consulta = consulta.split(" ")
    # Separando-se os termos da pesquisa, de forma a evitar a contagem de palavras repetidas. Por exemplo,
    # uma consulta por "thaynan andrey thaynan", evitaria que eu calculasse "thaynan" duas vezes.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            for id_doc in indices_invertidos[termo].keys():
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + 1
                else:
                    documentos_com_peso[id_doc] = 1
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *TF*

In [10]:
def get_documentos_ordem_TF(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Term Frequency Weighting",
        na qual, quanto mais o termo se repete na consulta ou no documento, mais pesos
        ele tem.
    '''
    consulta = consulta.split(" ")
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            
            for id_doc in indices_invertidos[termo].keys():
                peso_doc_termo = indices_invertidos[termo][id_doc]
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *TF-IDF*

In [11]:
def get_documentos_ordem_TF_IDF(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Term Frequency Weighting" acrescentado do IDF,
        no qual, termos muito populares no documento são penalizados e termos pouco populares são valorizados,
        o que evita que stop words se sobreponham a outras palavras.
    '''
    consulta = consulta.split(" ")
    quantidade_documentos_colecao = len(documentos)
    
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            quantidade_documentos_com_termo = len(indices_invertidos[termo].keys())
            
            for id_doc in indices_invertidos[termo].keys():
                peso_doc_termo = indices_invertidos[termo][id_doc] * idf(quantidade_documentos_colecao, quantidade_documentos_com_termo)
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *BM25*

In [45]:
def get_documentos_ordem_bm25(consulta, indices_invertidos, k):
    '''
        Obtém os documentos ordenados segundo a ideia do BM25.
    '''
    consulta = consulta.split(" ")
    quantidade_documentos_colecao = len(documentos)
    
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            quantidade_documentos_com_termo = len(indices_invertidos[termo].keys())
            
            for id_doc in indices_invertidos[termo].keys():
                numerador_peso_doc_termo = (k + 1) * indices_invertidos[termo][id_doc] * idf(quantidade_documentos_colecao, quantidade_documentos_com_termo)
                denominador_peso_doc_termo = indices_invertidos[termo][id_doc] + k
                peso_doc_termo = numerador_peso_doc_termo / denominador_peso_doc_termo
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k)
    
    return k_primeiros_id_documentos

### Q1: Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank

#### Escolhi a consulta "reforma da previdencia" e espero encontrar o documento da url abaixo, pois na mesma temos a discussão sobre os plano de Paulo Guedes para a reforma da previdência.
    
url: https://brasil.elpais.com/brasil/2019/01/03/economia/1546549523_759922.html

In [27]:
url_esperada = "https://brasil.elpais.com/brasil/2019/01/03/economia/1546549523_759922.html"

#### Consulta em **Representação Binária**

In [39]:
# Obtém o ranking top-10 para a consulta "reforma da previdencia" usando o algoritmo de Represetnação Binária.
reforma_previdencia_rep_binaria = get_documentos_ordem_representacao_binaria("reforma da previdencia",
                                                                             indices_invertidos_com_frequencia,
                                                                             10)

# Obtém as urls dos documentos advindos do ranking
urls_documentos_ordem_representacao_binaria = [data_frame["url"][i] for i in reforma_previdencia_rep_binaria]

# Obtém a posição, caso exista, da url no ranking
posicao_ranking_consulta_rep_binaria = urls_documentos_ordem_representacao_binaria.index(url_esperada) + 1

print "Posição do documento que espero no ranking de 'Representacao Binaria' é: %i" %posicao_ranking_consulta_rep_binaria

Posição do documento que espero no ranking de 'Representacao Binaria' é: 3


##### Avaliação para a consulta

Note que a url que eu esperava estava no ranking dos documentos para a consulta em 'Representação Binária'. Ainda mais apareceu como a terceira do ranking, o que a caracteriza com o Reciprocal Rank 1/3. Observa-se também que o primeiro e o segundo ranking também citam bastante sobre a reforma da previdência. Logo a consulta é de fato o que esperava.

#### Consulta em **TF**

In [38]:
# Obtém o ranking top-10 para a consulta "reforma da previdencia" usando o algoritmo de TF.
reforma_previdencia_tf = get_documentos_ordem_TF("reforma da previdencia",
                                                 indices_invertidos_com_frequencia,
                                                 10)

# Obtém as urls dos documentos advindos do ranking
urls_documentos_ordem_tf = [data_frame["url"][i] for i in reforma_previdencia_tf]

# Obtém a posição, caso exista, da url no ranking
posicao_ranking_consulta_tf = urls_documentos_ordem_tf.index(url_esperada) + 1

print "Posição do documento que espero no ranking de 'TF' é: %i" %posicao_ranking_consulta_tf

Posição do documento que espero no ranking de 'TF' é: 1


##### Avaliação para a consulta

Note que a url que eu esperava estava no ranking dos documentos para a consulta em 'TF'. Ainda mais apareceu como a primeira do ranking, o que a caracteriza com o Reciprocal Rank 1/1 = 1. Logo a consulta é de fato o que esperava, obtendo-se um retorno ótimo no ranking.

#### Consulta em **TF-IDF**

In [42]:
# Obtém o ranking top-10 para a consulta "reforma da previdencia" usando o algoritmo de TF-IDF.
reforma_previdencia_tf_idf = get_documentos_ordem_TF_IDF("reforma da previdencia",
                                                 indices_invertidos_com_frequencia,
                                                 10)

# Obtém as urls dos documentos advindos do ranking
urls_documentos_ordem_tf_idf = [data_frame["url"][i] for i in reforma_previdencia_tf_idf]

# Obtém a posição, caso exista, da url no ranking
posicao_ranking_consulta_tf_idf = urls_documentos_ordem_tf_idf.index(url_esperada) + 1

print "Posição do documento que espero no ranking de 'TF-IDF' é: %i" %posicao_ranking_consulta_tf_idf

Posição do documento que espero no ranking de 'TF-IDF' é: 1


##### Avaliação para a consulta

Note que a url que eu esperava estava no ranking dos documentos para a consulta em 'TF-IDF'. Ainda mais apareceu como a primeira do ranking, o que a caracteriza com o Reciprocal Rank 1/1 = 1. Logo a consulta é de fato o que esperava, obtendo-se um retorno ótimo no ranking.

#### Consulta em **BM25**

In [44]:
# Obtém o ranking top-10 para a consulta "reforma da previdencia" usando o algoritmo de BM25.
reforma_previdencia_bm25 = get_documentos_ordem_bm25("reforma da previdencia",
                                                 indices_invertidos_com_frequencia,
                                                 10)

# Obtém as urls dos documentos advindos do ranking
urls_documentos_ordem_bm25 = [data_frame["url"][i] for i in reforma_previdencia_bm25]

# Obtém a posição, caso exista, da url no ranking
posicao_ranking_consulta_bm25 = urls_documentos_ordem_bm25.index(url_esperada) + 1

print "Posição do documento que espero no ranking de 'BM25' é: %i" %posicao_ranking_consulta_bm25

Posição do documento que espero no ranking de 'BM25' é: 1


##### Avaliação para a consulta

Note que a url que eu esperava estava no ranking dos documentos para a consulta em 'BM25'. Ainda mais apareceu como a primeira do ranking, o que a caracteriza com o Reciprocal Rank 1/1 = 1. Logo a consulta é de fato o que esperava, obtendo-se um retorno ótimo no ranking.

### Q2: A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.

#### Representação Binária

#### TF

#### TF-IDF

#### BM25

### Q3: Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG.

#### Representação Binária

#### TF

#### TF-IDF

#### BM25